# Vanilla closed-form with pybind11

This notebook demonstrates the usage of a python extension module built upon **pybind11**. The following packages are required to get it work:
- pybind11
- numpy
- bqplot
- ipywidgets

We suggest to insall them with conda: `conda install pybind11 bqplot ipywidgets -c conda-forge`
so the dependencies are handled for you.

In [ ]:
import math
import numpy as np
import pybind_closed_forms as pcf
from bqplot import (LinearScale, Lines, Axis, Figure)
import ipywidgets as widgets
import ipyvolume as ipv
from IPython.display import display

## 1. Simple european call

In this section, we plot the price of a european call depending on the spot, and show how the volatility, maturity and rates
can influence this price curve. We can pass numpy arrays to the functions defined in `pybind_closed_forms` even if these are not python functions.

In [ ]:
iscall = True
vol = 0.2
mat = 1.
rate = 0.04
strike = 1.
spot = np.arange(0.1, 1.9, 0.01)

In [ ]:
# Documentation works as if it was a python function
?pcf.vanilla_discounted_payoff

In [ ]:
price = np.empty_like(spot)
discounted_payoff = np.empty_like(spot)
for i in range(spot.shape[0]):
    price[i] = pcf.bs_discounted_price(spot[i], strike, vol, mat, rate, iscall)
    discounted_payoff[i] = pcf.vanilla_discounted_payoff(spot[i], strike, mat, rate, iscall)

In [ ]:
sc_x = LinearScale()
sc_y = LinearScale(max=1.)
call_graph = Lines(x=spot, y=price, scales={'x': sc_x, 'y': sc_y}, labels=['Price'], display_legend=True)
payoff_graph = Lines(x=spot, y=discounted_payoff, scales={'x': sc_x, 'y': sc_y}, labels=['Payoff'], colors=['red'],
                    display_legend=True)
ax_x = Axis(scale=sc_x, label="spot")
ax_y = Axis(scale=sc_y, orientation='vertical', label="price")

vol_slider = widgets.FloatSlider(value=vol, min=0, max=1, step=0.05, description='volatility')
def handle_vol_change(change):
    global vol
    vol = change.new
    tmp_call_graph = np.empty_like(spot)
    for i in range(spot.shape[0]):
        tmp_call_graph[i] = pcf.bs_discounted_price(spot[i], strike, vol, mat, rate, iscall)
    call_graph.y = tmp_call_graph
vol_slider.observe(handle_vol_change, names='value')

rate_slider = widgets.FloatSlider(value=rate, min=0, max=0.1, step=0.01, description='rate')
def handle_rate_change(change):
    global rate
    rate = change.new
    tmp_call_graph = np.empty_like(spot)
    tmp_payoff_graph = np.empty_like(spot)
    for i in range(spot.shape[0]):
        tmp_call_graph[i] = pcf.bs_discounted_price(spot[i], strike, vol, mat, rate, iscall)
        tmp_payoff_graph[i] = pcf.vanilla_discounted_payoff(spot[i], strike, mat, rate, iscall)
    call_graph.y = tmp_call_graph
    payoff_graph.y = tmp_payoff_graph
rate_slider.observe(handle_rate_change, names='value')

mat_slider = widgets.FloatSlider(value=mat, min=0.5, max=10., step=0.5, description='maturity')
def handle_mat_change(change):
    global mat
    mat = change.new
    tmp_call_graph = np.empty_like(spot)
    tmp_payoff_graph = np.empty_like(spot)
    for i in range(spot.shape[0]):
        tmp_call_graph[i] = pcf.bs_discounted_price(spot[i], strike, vol, mat, rate, iscall)
        tmp_payoff_graph[i] = pcf.vanilla_discounted_payoff(spot[i], strike, mat, rate, iscall)
    call_graph.y = tmp_call_graph
    payoff_graph.y = tmp_payoff_graph
mat_slider.observe(handle_mat_change, names='value')

figure = Figure(marks=[call_graph, payoff_graph], axes=[ax_x, ax_y], title='European Call',
               legend_location='top-left')

r = widgets.VBox([figure, vol_slider, rate_slider, mat_slider])
display(r)

In [ ]:
payoff = pcf.VanillaPayoff(1., 1., True)
pr = pcf.vanilla_discounted_payoff(spot[150], rate, payoff)
pr